In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import datetime
import time
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectPercentile,f_regression

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_1 = pd.read_csv('./data/train_1.csv')
train_2 = pd.read_csv('./data/train_2.csv')
train_3 = pd.read_csv('./data/train_3.csv')
train_4 = pd.read_csv('./data/train_4.csv')

test_1 = pd.read_csv('./data/test_1.csv')
test_2 = pd.read_csv('./data/test_2.csv')
test_3 = pd.read_csv('./data/test_3.csv')
test_4 = pd.read_csv('./data/test_4.csv')

In [3]:
data_1 = pd.concat([train_1,test_1],axis=0,ignore_index=True,sort=False)
data_2 = pd.concat([train_2,test_2],axis=0,ignore_index=True,sort=False)
data_3 = pd.concat([train_3,test_3],axis=0,ignore_index=True,sort=False)
data_4 = pd.concat([train_4,test_4],axis=0,ignore_index=True,sort=False)

In [4]:
# 返回时间戳
def get_timestamp(x):
    try:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f")
        t = int(round((t.timestamp())))  # 四舍五入，去掉毫秒级  
    except:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
        t = int(t.timestamp())
    return t   

In [5]:
data_1['时间戳'] = data_1['时间'].map(lambda x: get_timestamp(x))
data_2['时间戳'] = data_2['时间'].map(lambda x: get_timestamp(x))
data_3['时间戳'] = data_3['时间'].map(lambda x: get_timestamp(x))
data_4['时间戳'] = data_4['时间'].map(lambda x: get_timestamp(x))

In [6]:
# 将带有毫秒级时间四舍五入为秒级时间
# 输入字符串时间，"%Y-%m-%d %H:%M:%S.%f"
# 输出datetime格式的时间，年月日时分秒

def get_new_time(x):
    try:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f")
        t = round(t.timestamp())  # 四舍五入，去掉毫秒级
        t = datetime.datetime.fromtimestamp(t)  # 转换为datetime格式
    except:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")  # 转换为datetime格式
    return t

In [7]:

data_1['month'] = data_1['时间'].map(lambda x: get_new_time(x).month)
data_1['day'] = data_1['时间'].map(lambda x: get_new_time(x).day)
data_1['hour'] = data_1['时间'].map(lambda x: get_new_time(x).hour)
data_1['minute'] = data_1['时间'].map(lambda x: get_new_time(x).minute)

data_2['month'] = data_2['时间'].map(lambda x: get_new_time(x).month)
data_2['day'] = data_2['时间'].map(lambda x: get_new_time(x).day)
data_2['hour'] = data_2['时间'].map(lambda x: get_new_time(x).hour)
data_2['minute'] = data_2['时间'].map(lambda x: get_new_time(x).minute)

data_3['month'] = data_3['时间'].map(lambda x: get_new_time(x).month)
data_3['day'] = data_3['时间'].map(lambda x: get_new_time(x).day)
data_3['hour'] = data_3['时间'].map(lambda x: get_new_time(x).hour)
data_3['minute'] = data_3['时间'].map(lambda x: get_new_time(x).minute)

data_4['month'] = data_4['时间'].map(lambda x: get_new_time(x).month)
data_4['day'] = data_4['时间'].map(lambda x: get_new_time(x).day)
data_4['hour'] = data_4['时间'].map(lambda x: get_new_time(x).hour)
data_4['minute'] = data_4['时间'].map(lambda x: get_new_time(x).minute)

In [8]:
data_1['风向_count']= data_1['风向'].map(data_1['风向'].value_counts()).astype(int)
data_1['风向_count'] = (data_1['风向_count']-data_1['风向_count'].min()) / (data_1['风向_count'].max()-data_1['风向_count'].min())

data_2['风向_count']= data_2['风向'].map(data_2['风向'].value_counts()).astype(int)
data_2['风向_count'] = (data_2['风向_count']-data_2['风向_count'].min()) / (data_2['风向_count'].max()-data_2['风向_count'].min())

data_3['风向_count']= data_3['风向'].map(data_3['风向'].value_counts()).astype(int)
data_3['风向_count'] = (data_3['风向_count']-data_3['风向_count'].min()) / (data_3['风向_count'].max()-data_3['风向_count'].min())

data_4['风向_count']= data_4['风向'].map(data_4['风向'].value_counts()).astype(int)
data_4['风向_count'] = (data_4['风向_count']-data_4['风向_count'].min()) / (data_4['风向_count'].max()-data_4['风向_count'].min())

In [9]:
# 距离中心13点30分(13.5)的时间距离
data_1['dis2peak'] = data_1['时间'].map(lambda x: (13.5-abs(13.5-get_new_time(x).hour))/13.5)
data_2['dis2peak'] = data_2['时间'].map(lambda x: (13.5-abs(13.5-get_new_time(x).hour))/13.5)
data_3['dis2peak'] = data_3['hour'].map(lambda x: (13.5-abs(13.5-x))/13.5)
data_4['dis2peak'] = data_4['时间'].map(lambda x: (13.5-abs(13.5-get_new_time(x).hour))/13.5)

In [10]:
def map_hour(h):
    if h < 7:
        return 1
    elif h < 10:
        return 2
    elif h < 13:
        return 3
    elif h < 16:
        return 4
    elif h < 19:
        return 5
    else:
        return 1

In [11]:
data_1['maphour'] = data_1['时间'].map(lambda x: map_hour(get_new_time(x).hour))
data_2['maphour'] = data_2['时间'].map(lambda x:map_hour(get_new_time(x).hour))
data_3['maphour'] = data_3['hour'].map(map_hour)
data_4['maphour'] = data_4['时间'].map(lambda x:map_hour(get_new_time(x).hour))

In [12]:
yuanshi_feature = ['时间', '辐照度', '风速', '风向', '温度', '压强', '湿度', '实发辐照度', '实际功率', 'id','时间戳',
       'month', 'day', 'hour', '风向_count','dis2peak']

萌鸡、新添特征

In [13]:
data_1.columns

Index(['时间', '辐照度', '风速', '风向', '温度', '压强', '湿度', '实发辐照度', '实际功率', 'id', '时间戳',
       'month', 'day', 'hour', 'minute', '风向_count', 'dis2peak', 'maphour'],
      dtype='object')

In [14]:
for data in [data_1,data_2,data_3,data_4]:
    data['time_2_hours_flag'] = data['时间戳']/120000000
    data['time_2_hours_flag'] = data['time_2_hours_flag'].round()
    
    data['time_1flag'] = data['时间戳']/10000000
    data['time_1flag'] = data['time_1flag'].round()
    data['time_5flag'] = data['时间戳']/5000000
    data['time_5flag'] = data['time_5flag'].round()
    data['time_10flag'] = data['时间戳']/1000000
    data['time_10flag'] = data['time_10flag'].round()
for i in ['time_2_hours_flag','time_1flag', 'time_5flag', 'time_10flag']:
    for j in ['辐照度','风速','温度','压强','湿度','风向']:
        temp1 = data_1.groupby(i, as_index = False)[j].agg({i+'_'+j+'_mean':'mean', i+'_'+j+'_max':'max',i+'_'+j+'_min':'min',i+'_'+j+'_var':'var',i+'_'+j+'_median':'median'})
        temp2 = data_2.groupby(i, as_index = False)[j].agg({i+'_'+j+'_mean':'mean', i+'_'+j+'_max':'max',i+'_'+j+'_min':'min',i+'_'+j+'_var':'var',i+'_'+j+'_median':'median'})
        temp3 = data_3.groupby(i, as_index = False)[j].agg({i+'_'+j+'_mean':'mean', i+'_'+j+'_max':'max',i+'_'+j+'_min':'min',i+'_'+j+'_var':'var',i+'_'+j+'_median':'median'})
        temp4 = data_4.groupby(i, as_index = False)[j].agg({i+'_'+j+'_mean':'mean', i+'_'+j+'_max':'max',i+'_'+j+'_min':'min',i+'_'+j+'_var':'var',i+'_'+j+'_median':'median'})
        data_1 = pd.merge(data_1, temp1, on=i, how='left')
        data_2 = pd.merge(data_2, temp2, on=i, how='left')
        data_3 = pd.merge(data_3, temp3, on=i, how='left')
        data_4 = pd.merge(data_4, temp4, on=i, how='left')
        
for i in ['辐照度','风速','温度','压强','湿度','风向']:
#     col = str('time_1flag_'+i+'_mean')
    data_1[i+'-mean_2_hours'] = data_1[i] - data_1['time_2_hours_flag_'+i+'_mean']   
    data_2[i+'-mean_2_hours'] = data_2[i] - data_2['time_2_hours_flag_'+i+'_mean'] 
    data_3[i+'-mean_2_hours'] = data_3[i] - data_3['time_2_hours_flag_'+i+'_mean'] 
    data_4[i+'-mean_2_hours'] = data_4[i] - data_4['time_2_hours_flag_'+i+'_mean'] 
    
for i in ['辐照度','风速','温度','压强','湿度','风向']:
#     col = str('time_1flag_'+i+'_mean')
    data_1[i+'-mean_everyday'] = data_1[i] - data_1['time_1flag_'+i+'_mean']   
    data_2[i+'-mean_everyday'] = data_2[i] - data_2['time_1flag_'+i+'_mean'] 
    data_3[i+'-mean_everyday'] = data_3[i] - data_3['time_1flag_'+i+'_mean'] 
    data_4[i+'-mean_everyday'] = data_4[i] - data_4['time_1flag_'+i+'_mean'] 
for i in ['辐照度','风速','温度','压强','湿度','风向']:
#     col = str('time_1flag_'+i+'_mean')
    data_1[i+'-mean_5day'] = data_1[i] - data_1['time_5flag_'+i+'_mean']   
    data_2[i+'-mean_5day'] = data_2[i] - data_2['time_5flag_'+i+'_mean'] 
    data_3[i+'-mean_5day'] = data_3[i] - data_3['time_5flag_'+i+'_mean'] 
    data_4[i+'-mean_5day'] = data_4[i] - data_4['time_5flag_'+i+'_mean'] 

for i in ['辐照度','风速','温度','压强','湿度','风向']:
#     col = str('time_1flag_'+i+'_mean')
    data_1[i+'_1f-5f_everyweek'] = data_1['time_1flag_'+i+'_mean'] - data_1['time_5flag_'+i+'_mean']   
    data_2[i+'_1f-5f_everyweek'] = data_2['time_1flag_'+i+'_mean'] - data_2['time_5flag_'+i+'_mean'] 
    data_3[i+'_1f-5f_everyweek'] = data_3['time_1flag_'+i+'_mean'] - data_3['time_5flag_'+i+'_mean'] 
    data_4[i+'_1f-5f_everyweek'] = data_4['time_1flag_'+i+'_mean'] - data_4['time_5flag_'+i+'_mean']     
    
for i in ['辐照度','风速','温度','压强','湿度','风向']:
#     col = str('time_1flag_'+i+'_mean')
    data_1[i+'-mean_everyweek'] = data_1['time_1flag_'+i+'_mean'] - data_1['time_10flag_'+i+'_mean']   
    data_2[i+'-mean_everyweek'] = data_2['time_1flag_'+i+'_mean'] - data_2['time_10flag_'+i+'_mean'] 
    data_3[i+'-mean_everyweek'] = data_3['time_1flag_'+i+'_mean'] - data_3['time_10flag_'+i+'_mean'] 
    data_4[i+'-mean_everyweek'] = data_4['time_1flag_'+i+'_mean'] - data_4['time_10flag_'+i+'_mean'] 

In [15]:
def math(data,i,j):
    data[items[i]+'+'+items[j]] = data_1[items[i]] + data_1[items[j]]
    data[items[i]+'*'+items[j]] = data_1[items[i]] + data_1[items[j]]
    data[items[i]+'方+'+items[j]] = data_1[items[i]]*data_1[items[i]] + data_1[items[j]]*data_1[items[j]]
    return data
   
items = ['辐照度','风速','温度','压强','湿度','风向']
for i in range(len(items)):
    for j in range(i+1, len(items)):
        data_1 = math(data_1,i,j)
        data_2 = math(data_2,i,j)
        data_3 = math(data_3,i,j)
        data_4 = math(data_4,i,j)

In [16]:
def cha1(data):    
    for i in ['辐照度','风速','温度','压强','湿度','风向','温度*压强','风速*温度']:
        data[i+'_1'] = data[i].shift(1)
        data[i+'_1'].fillna(-1,inplace =True)
        data[i+'_差1'] = data[i] - data[i+'_1']
    return data
def cha3(data):    
    for i in ['辐照度','风速','温度','压强','湿度','风向','温度*压强','风速*温度']:
        data[i+'_3'] = data[i].shift(3)
        data[i+'_3'].fillna(-1,inplace =True)
        data[i+'_差3'] = data[i] - data[i+'_3']
    return data
for i in [data_1,data_2,data_3,data_4]:
    cha1(i)
    cha3(i)
    

In [17]:
def get_ab_month(x):
    return abs(x-6)   
data_1['ab_month'] = data_1['month'].map(lambda x: get_ab_month(x))
data_2['ab_month'] = data_2['month'].map(lambda x: get_ab_month(x))
data_3['ab_month'] = data_3['month'].map(lambda x: get_ab_month(x))
data_4['ab_month'] = data_4['month'].map(lambda x: get_ab_month(x))

def get_ab_hour(x):
    return abs(x-12)   
data_1['abs_hour'] = data_1['hour'].map(lambda x: get_ab_hour(x))
data_2['abs_hour'] = data_2['hour'].map(lambda x: get_ab_hour(x))
data_3['abs_hour'] = data_3['hour'].map(lambda x: get_ab_hour(x))
data_4['abs_hour'] = data_4['hour'].map(lambda x: get_ab_hour(x))

data_1['abs_时间戳'] = data_1['时间戳']%31536000
data_2['abs_时间戳'] = data_2['时间戳']%31536000
data_3['abs_时间戳'] = data_3['时间戳']%31536000
data_4['abs_时间戳'] = data_4['时间戳']%31536000

In [18]:
a = data_1.columns.tolist()
a

['时间',
 '辐照度',
 '风速',
 '风向',
 '温度',
 '压强',
 '湿度',
 '实发辐照度',
 '实际功率',
 'id',
 '时间戳',
 'month',
 'day',
 'hour',
 'minute',
 '风向_count',
 'dis2peak',
 'maphour',
 'time_2_hours_flag',
 'time_1flag',
 'time_5flag',
 'time_10flag',
 'time_2_hours_flag_辐照度_mean',
 'time_2_hours_flag_辐照度_max',
 'time_2_hours_flag_辐照度_min',
 'time_2_hours_flag_辐照度_var',
 'time_2_hours_flag_辐照度_median',
 'time_2_hours_flag_风速_mean',
 'time_2_hours_flag_风速_max',
 'time_2_hours_flag_风速_min',
 'time_2_hours_flag_风速_var',
 'time_2_hours_flag_风速_median',
 'time_2_hours_flag_温度_mean',
 'time_2_hours_flag_温度_max',
 'time_2_hours_flag_温度_min',
 'time_2_hours_flag_温度_var',
 'time_2_hours_flag_温度_median',
 'time_2_hours_flag_压强_mean',
 'time_2_hours_flag_压强_max',
 'time_2_hours_flag_压强_min',
 'time_2_hours_flag_压强_var',
 'time_2_hours_flag_压强_median',
 'time_2_hours_flag_湿度_mean',
 'time_2_hours_flag_湿度_max',
 'time_2_hours_flag_湿度_min',
 'time_2_hours_flag_湿度_var',
 'time_2_hours_flag_湿度_median',
 'time_2_hours_flag_风向

In [19]:
# # 这组特征没卵用
# for i in ['辐照度','风速','温度','压强','湿度','风向']:
# #     col = str('time_1flag_'+i+'_mean')
#     data_1[i+'_diff_everyday'] = data_1['time_1flag_'+i+'_max'] - data_1['time_1flag_'+i+'_min']   
#     data_2[i+'_diff_everyday'] = data_2['time_1flag_'+i+'_max'] - data_2['time_1flag_'+i+'_min'] 
#     data_3[i+'_diff_everyday'] = data_3['time_1flag_'+i+'_max'] - data_3['time_1flag_'+i+'_min'] 
#     data_4[i+'_diff_everyday'] = data_4['time_1flag_'+i+'_max'] - data_4['time_1flag_'+i+'_min'] 

In [20]:
# new_feature_cols = ['辐照度-mean_everyday','湿度-mean_everyday','风向-mean_everyday','压强-mean_everyday','温度-mean_everyday',
#                     '风速-mean_everyday','辐照度-mean_everyweek','time_10flag_辐照度_mean',]

new_feature_cols = ['辐照度-mean_everyday','湿度-mean_everyday','风向-mean_everyday','压强-mean_everyday','温度-mean_everyday',
                    '风速-mean_everyday','辐照度-mean_everyweek','time_10flag_辐照度_mean','风速*温度','温度*压强','ab_month'
                   ,'abs_hour', 
'辐照度_差1',
 '风速_差1',
 '温度_差1',
 '压强_差1',
 '湿度_差1',
 '风向_差1']
# '辐照度_差3',
#  '风速_差3',
#  '温度_差3',
#  '压强_差3',
#  '湿度_差3',
#  '风向_差3'




In [21]:

#  '辐照度_差1',
#  '风速_差1',
#  '温度_差1',
#  '压强_差1',
#  '湿度_差1',
#  '风向_差1',
#  '辐照度_差3',
#  '风速_差3',
#  '温度_差3',
#  '压强_差3',
#  '湿度_差3',
#  '风向_差3',

In [22]:
# X = data_1
# import ssl
# def count_corr(df):
#     '''
#     输入dataframe
#     输出相关系数dataframe:col_1,col_2,cor(不包含同一特征且已去重复)
#     '''
# #     计算列之间的相关系数，abs求绝对值，unstack变成堆叠形式的，sort_values降序排列，然后用reset_index重新赋值索引
#     x = df.corr().abs().unstack().sort_values(ascending=False).reset_index()
# #     找到除了对角线之外的行
#     x = x.loc[x.level_0!=x.level_1]
# #
#     x2 = pd.DataFrame([sorted(i) for i in x[['level_0','level_1']].values])
#     x2['cor'] = x[0].values
#     x2.columns = ['col_1','col_2','cor']
# #     删除重复的行
#     return x2.drop_duplicates()
# v = X.var()
# constant_feature = v[v==0].index.values.tolist()

# for i in constant_feature:
#     _ = X.pop(i)

# train_corr_col = count_corr(X)
# corr1_col = train_corr_col[train_corr_col.cor>0.9].col_2.values.tolist()
# corr_col = list(set(corr1_col))
# print(corr_col)
# # 删除掉相关性大于0.99的列
# for i in corr_col:
#     if i not in['时间','实发辐照度','实际功率','id']:
#         _ = X.pop(i)



In [23]:
# a = X.columns.tolist()
# a

In [24]:
col= yuanshi_feature + new_feature_cols
# col = a
data_1 = data_1[col]
data_2 = data_2[col]
data_3 = data_3[col]
data_4 = data_4[col]


萌鸡、至此

In [25]:
X_train_1 = data_1[data_1['实际功率'].notnull()].drop(['时间','实发辐照度','实际功率','id'],axis=1)
y_train_1 = data_1[data_1['实际功率'].notnull()]['实际功率']
X_test_1 = data_1[data_1['实际功率'].isnull()].drop(['时间','实发辐照度','实际功率','id'],axis=1)

In [26]:
train_1_features = X_train_1.columns
train_1_features

Index(['辐照度', '风速', '风向', '温度', '压强', '湿度', '时间戳', 'month', 'day', 'hour',
       '风向_count', 'dis2peak', '辐照度-mean_everyday', '湿度-mean_everyday',
       '风向-mean_everyday', '压强-mean_everyday', '温度-mean_everyday',
       '风速-mean_everyday', '辐照度-mean_everyweek', 'time_10flag_辐照度_mean',
       '风速*温度', '温度*压强', 'ab_month', 'abs_hour', '辐照度_差1', '风速_差1', '温度_差1',
       '湿度_差1', '风向_差1', '温度*压强_差1', '风速*温度_差1'],
      dtype='object')

In [27]:
prediction_1 = test_1[['id']]

X_train_1_loc = X_train_1.values
X_test_1_loc = X_test_1.values
y_train_1_loc = y_train_1.values

# SKB = SelectPercentile(f_regression,percentile=95).fit(X_train_1_loc,y_train_1_loc)
# X_train_1_loc = SKB.transform(X_train_1_loc)
# X_test_1_loc = SKB.transform(X_test_1_loc)

In [28]:
from sklearn.metrics import mean_absolute_error
def my_metric_1(y_true,y_pred): 
    y_true_1 = y_true[y_true>=10*0.03]
    y_pred_1 = y_pred[y_true>=10*0.03]
    score = mean_absolute_error(y_true_1,y_pred_1)/10.0
    return 'my_metric_1',score,False  # False表示score不是越大越好(is_bigger_better)

In [29]:
model_1 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=100,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.001,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)


skf=list(StratifiedKFold(y_train_1_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_1.fit(X_train_1_loc[train_index], y_train_1_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_1_loc[train_index], y_train_1_loc[train_index]), 
                                    (X_train_1_loc[test_index], y_train_1_loc[test_index])],
                          eval_metric=my_metric_1,
                          early_stopping_rounds=50)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_1'])
    loss += lgb_model.best_score_['valid']['my_metric_1']
    test_pred= lgb_model.predict(X_test_1_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_1['predict_%s' % str(i)] = test_pred
print('my_metric_1:', baseloss, loss/10)

p = prediction_1.drop(['id'],axis=1)
prediction_1['prediction']=p.mean(axis=1)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 8.17936	train's my_metric_1: 0.294849	valid's l2: 8.0235	valid's my_metric_1: 0.292212
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 7.47853	train's my_metric_1: 0.282267	valid's l2: 7.33989	valid's my_metric_1: 0.279914
[3]	train's l2: 6.84593	train's my_metric_1: 0.270497	valid's l2: 6.7233	valid's my_metric_1: 0.268414
[4]	train's l2: 6.27512	train's my_metric_1: 0.259485	valid's l2: 6.16889	valid's my_metric_1: 0.257718
[5]	train's l2: 5.75828	train's my_metric_1: 0.249115	valid's l2: 5.665	valid's my_metric_1: 0.247597
[6]	train's l2: 5.29023	train's my_metric_1: 0.239447	valid's l2: 5.21086	valid's my_metric_1: 0.238118
[7]	train's l2: 4.86746	train's my_metric_1: 0.230314	valid's l2: 4.79926	valid's my_metric_1: 0.229108
[8]	train's l2: 4.48547	train's my_metric_1: 0.221758	valid's l2: 4.42577	valid's my_metric_1: 0.220649
[9]	train's l2: 4.13993	train's my_metric_1: 0.213747	valid's l2: 4.08963	valid's my_metric_1: 0.212739

[83]	train's l2: 0.606986	train's my_metric_1: 0.0868193	valid's l2: 0.69722	valid's my_metric_1: 0.0921775
[84]	train's l2: 0.603211	train's my_metric_1: 0.0865246	valid's l2: 0.69435	valid's my_metric_1: 0.0919368
[85]	train's l2: 0.599469	train's my_metric_1: 0.0862295	valid's l2: 0.691262	valid's my_metric_1: 0.0916917
[86]	train's l2: 0.595753	train's my_metric_1: 0.0859136	valid's l2: 0.68742	valid's my_metric_1: 0.0913916
[87]	train's l2: 0.593039	train's my_metric_1: 0.0857003	valid's l2: 0.685479	valid's my_metric_1: 0.0912388
[88]	train's l2: 0.590037	train's my_metric_1: 0.0854589	valid's l2: 0.683476	valid's my_metric_1: 0.0910423
[89]	train's l2: 0.587431	train's my_metric_1: 0.0852062	valid's l2: 0.681941	valid's my_metric_1: 0.0908471
[90]	train's l2: 0.585385	train's my_metric_1: 0.0849868	valid's l2: 0.680293	valid's my_metric_1: 0.0906663
[91]	train's l2: 0.582825	train's my_metric_1: 0.0847316	valid's l2: 0.678076	valid's my_metric_1: 0.0904709
[92]	train's l2: 0.579

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold 1
[1]	train's l2: 8.16217	train's my_metric_1: 0.294358	valid's l2: 8.14716	valid's my_metric_1: 0.298177
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 7.46336	train's my_metric_1: 0.281832	valid's l2: 7.44802	valid's my_metric_1: 0.285427
[3]	train's l2: 6.83233	train's my_metric_1: 0.270094	valid's l2: 6.81767	valid's my_metric_1: 0.273467
[4]	train's l2: 6.26147	train's my_metric_1: 0.259089	valid's l2: 6.24469	valid's my_metric_1: 0.262155
[5]	train's l2: 5.74559	train's my_metric_1: 0.248753	valid's l2: 5.73121	valid's my_metric_1: 0.25166
[6]	train's l2: 5.27978	train's my_metric_1: 0.239074	valid's l2: 5.26283	valid's my_metric_1: 0.241792
[7]	train's l2: 4.85819	train's my_metric_1: 0.229952	valid's l2: 4.84405	valid's my_metric_1: 0.232616
[8]	train's l2: 4.47761	train's my_metric_1: 0.221487	valid's l2: 4.463	valid's my_metric_1: 0.223982
[9]	train's l2: 4.13344	train's my_metric_1: 0.213549	valid's l2: 4.11827	valid's my_metric_1: 0.21596

...

[1]	train's l2: 8.12284	train's my_metric_1: 0.29399	valid's l2: 8.50582	valid's my_metric_1: 0.30535
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 7.42794	train's my_metric_1: 0.281446	valid's l2: 7.77817	valid's my_metric_1: 0.292348
[3]	train's l2: 6.80006	train's my_metric_1: 0.2697	valid's l2: 7.12329	valid's my_metric_1: 0.280219
[4]	train's l2: 6.23318	train's my_metric_1: 0.258685	valid's l2: 6.52628	valid's my_metric_1: 0.268709
[5]	train's l2: 5.71996	train's my_metric_1: 0.248351	valid's l2: 5.98825	valid's my_metric_1: 0.257933
[6]	train's l2: 5.25665	train's my_metric_1: 0.238696	valid's l2: 5.50573	valid's my_metric_1: 0.247904
[7]	train's l2: 4.83701	train's my_metric_1: 0.2296	valid's l2: 5.06579	valid's my_metric_1: 0.238403
[8]	train's l2: 4.45767	train's my_metric_1: 0.221117	valid's l2: 4.67101	valid's my_metric_1: 0.229517
[9]	train's l2: 4.11439	train's my_metric_1: 0.213178	valid's l2: 4.30994	valid's my_metric_1: 0.221097
[10]	tra

[81]	train's l2: 0.610154	train's my_metric_1: 0.0871264	valid's l2: 0.709065	valid's my_metric_1: 0.0930562
[82]	train's l2: 0.605938	train's my_metric_1: 0.0867729	valid's l2: 0.705403	valid's my_metric_1: 0.0927763
[83]	train's l2: 0.602588	train's my_metric_1: 0.086496	valid's l2: 0.702416	valid's my_metric_1: 0.0925378
[84]	train's l2: 0.599693	train's my_metric_1: 0.086239	valid's l2: 0.700555	valid's my_metric_1: 0.092343
[85]	train's l2: 0.596617	train's my_metric_1: 0.0859573	valid's l2: 0.698166	valid's my_metric_1: 0.0921145
[86]	train's l2: 0.593916	train's my_metric_1: 0.0857252	valid's l2: 0.696098	valid's my_metric_1: 0.0919393
[87]	train's l2: 0.591173	train's my_metric_1: 0.0854906	valid's l2: 0.694289	valid's my_metric_1: 0.0917411
[88]	train's l2: 0.588099	train's my_metric_1: 0.0852388	valid's l2: 0.69205	valid's my_metric_1: 0.0915481
[89]	train's l2: 0.585069	train's my_metric_1: 0.0849825	valid's l2: 0.688923	valid's my_metric_1: 0.0912933
[90]	train's l2: 0.5814

[66]	train's l2: 0.672662	train's my_metric_1: 0.0926418	valid's l2: 0.771238	valid's my_metric_1: 0.096956
[67]	train's l2: 0.668004	train's my_metric_1: 0.0922275	valid's l2: 0.766855	valid's my_metric_1: 0.0965988
[68]	train's l2: 0.663479	train's my_metric_1: 0.0918202	valid's l2: 0.763283	valid's my_metric_1: 0.0962521
[69]	train's l2: 0.658132	train's my_metric_1: 0.0913667	valid's l2: 0.757569	valid's my_metric_1: 0.0958165
[70]	train's l2: 0.65365	train's my_metric_1: 0.0910149	valid's l2: 0.753131	valid's my_metric_1: 0.0954953
[71]	train's l2: 0.648975	train's my_metric_1: 0.0906095	valid's l2: 0.748275	valid's my_metric_1: 0.0951063
[72]	train's l2: 0.644559	train's my_metric_1: 0.090239	valid's l2: 0.744274	valid's my_metric_1: 0.0947966
[73]	train's l2: 0.640615	train's my_metric_1: 0.0899204	valid's l2: 0.740605	valid's my_metric_1: 0.0945437
[74]	train's l2: 0.637332	train's my_metric_1: 0.0896015	valid's l2: 0.737506	valid's my_metric_1: 0.0942597
[75]	train's l2: 0.632

[58]	train's l2: 0.718929	train's my_metric_1: 0.0962595	valid's l2: 0.786595	valid's my_metric_1: 0.100807
[59]	train's l2: 0.712118	train's my_metric_1: 0.0956918	valid's l2: 0.780464	valid's my_metric_1: 0.100302
[60]	train's l2: 0.705714	train's my_metric_1: 0.0951868	valid's l2: 0.775606	valid's my_metric_1: 0.0999073
[61]	train's l2: 0.699182	train's my_metric_1: 0.094653	valid's l2: 0.770045	valid's my_metric_1: 0.0994333
[62]	train's l2: 0.693414	train's my_metric_1: 0.0941699	valid's l2: 0.764875	valid's my_metric_1: 0.0990265
[63]	train's l2: 0.687121	train's my_metric_1: 0.0936691	valid's l2: 0.758447	valid's my_metric_1: 0.0985206
[64]	train's l2: 0.681744	train's my_metric_1: 0.0932122	valid's l2: 0.753377	valid's my_metric_1: 0.098097
[65]	train's l2: 0.676596	train's my_metric_1: 0.092785	valid's l2: 0.749491	valid's my_metric_1: 0.0977484
[66]	train's l2: 0.672048	train's my_metric_1: 0.0923903	valid's l2: 0.745532	valid's my_metric_1: 0.0974166
[67]	train's l2: 0.66775

[45]	train's l2: 0.837748	train's my_metric_1: 0.104887	valid's l2: 0.931345	valid's my_metric_1: 0.110104
[46]	train's l2: 0.825217	train's my_metric_1: 0.104038	valid's l2: 0.919802	valid's my_metric_1: 0.109307
[47]	train's l2: 0.813027	train's my_metric_1: 0.103195	valid's l2: 0.908863	valid's my_metric_1: 0.108536
[48]	train's l2: 0.802424	train's my_metric_1: 0.102415	valid's l2: 0.899206	valid's my_metric_1: 0.107795
[49]	train's l2: 0.79191	train's my_metric_1: 0.101674	valid's l2: 0.889211	valid's my_metric_1: 0.107071
[50]	train's l2: 0.781873	train's my_metric_1: 0.100971	valid's l2: 0.878677	valid's my_metric_1: 0.106357
[51]	train's l2: 0.772717	train's my_metric_1: 0.100275	valid's l2: 0.870717	valid's my_metric_1: 0.105741
[52]	train's l2: 0.763964	train's my_metric_1: 0.099625	valid's l2: 0.863271	valid's my_metric_1: 0.105128
[53]	train's l2: 0.755009	train's my_metric_1: 0.0989791	valid's l2: 0.856111	valid's my_metric_1: 0.104574
[54]	train's l2: 0.74647	train's my_m

In [30]:
train_features = X_train_1.columns
model_1.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_1.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
6,时间戳,466
8,day,376
12,辐照度-mean_everyday,327
5,湿度,271
21,温度*压强,240
18,辐照度-mean_everyweek,238
19,time_10flag_辐照度_mean,230
13,湿度-mean_everyday,230
2,风向,212
3,温度,209


In [31]:
train_features = X_train_1.columns
model_1.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_1.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
6,时间戳,466
8,day,376
12,辐照度-mean_everyday,327
5,湿度,271
21,温度*压强,240
18,辐照度-mean_everyweek,238
19,time_10flag_辐照度_mean,230
13,湿度-mean_everyday,230
2,风向,212
3,温度,209


In [32]:
X_train_2 = data_2[:len(train_2)].drop(['时间','实发辐照度','实际功率','id'],axis=1)

y_train_2 = data_2[:len(train_2)]['实际功率']

X_test_2 = data_2[len(train_2):].drop(['时间','实发辐照度','实际功率','id'],axis=1)

In [33]:
from sklearn.metrics import mean_absolute_error
def my_metric_2(y_true,y_pred): 
    y_true_2 = y_true[y_true>=10*0.03]
    y_pred_2 = y_pred[y_true>=10*0.03]
    score = mean_absolute_error(y_true_2,y_pred_2)/10.0
    return 'my_metric_2',score,False  # False表示score不是越大越好(is_bigger_better)

In [34]:
prediction_2 = test_2[['id']]

X_train_2_loc = X_train_2.values
X_test_2_loc = X_test_2.values
y_train_2_loc = y_train_2.values

model_2 = lgb.LGBMRegressor('gbdt',num_leaves=41,max_depth=-1,learning_rate=0.05,n_estimators=200,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.01,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_train_2_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_2.fit(X_train_2_loc[train_index], y_train_2_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_2_loc[train_index], y_train_2_loc[train_index]), 
                                    (X_train_2_loc[test_index], y_train_2_loc[test_index])],
                          eval_metric=my_metric_2,
                          early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_2'])
    loss += lgb_model.best_score_['valid']['my_metric_2']
    test_pred= lgb_model.predict(X_test_2_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_2['predict_%s' % str(i)] = test_pred
print('mae:', baseloss, loss/10)

p = prediction_2.drop(['id'],axis=1)
prediction_2['prediction']=p.mean(axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 7.38162	train's my_metric_2: 0.294946	valid's l2: 7.71642	valid's my_metric_2: 0.293716
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 6.78275	train's my_metric_2: 0.283273	valid's l2: 7.09347	valid's my_metric_2: 0.282188
[3]	train's l2: 6.24027	train's my_metric_2: 0.272368	valid's l2: 6.5314	valid's my_metric_2: 0.271468
[4]	train's l2: 5.75036	train's my_metric_2: 0.262138	valid's l2: 6.02373	valid's my_metric_2: 0.261466
[5]	train's l2: 5.30576	train's my_metric_2: 0.252481	valid's l2: 5.56508	valid's my_metric_2: 0.252114
[6]	train's l2: 4.90486	train's my_metric_2: 0.243446	valid's l2: 5.15125	valid's my_metric_2: 0.243373
[7]	train's l2: 4.54076	train's my_metric_2: 0.234883	valid's l2: 4.77405	valid's my_metric_2: 0.23504
[8]	train's l2: 4.20985	train's my_metric_2: 0.226886	valid's l2: 4.43065	valid's my_metric_2: 0.227368
[9]	train's l2: 3.91297	train's my_metric_2: 0.219494	valid's l2: 4.12507	valid's my_metric_2: 0.220

[94]	train's l2: 0.590278	train's my_metric_2: 0.0878148	valid's l2: 0.753805	valid's my_metric_2: 0.094446
[95]	train's l2: 0.587655	train's my_metric_2: 0.0875754	valid's l2: 0.751797	valid's my_metric_2: 0.0942869
[96]	train's l2: 0.584556	train's my_metric_2: 0.0873059	valid's l2: 0.749414	valid's my_metric_2: 0.094107
[97]	train's l2: 0.582309	train's my_metric_2: 0.0870931	valid's l2: 0.747473	valid's my_metric_2: 0.0939436
[98]	train's l2: 0.579657	train's my_metric_2: 0.0868499	valid's l2: 0.744921	valid's my_metric_2: 0.093726
[99]	train's l2: 0.577424	train's my_metric_2: 0.0866329	valid's l2: 0.743218	valid's my_metric_2: 0.0935774
[100]	train's l2: 0.573728	train's my_metric_2: 0.0863498	valid's l2: 0.739985	valid's my_metric_2: 0.093378
[101]	train's l2: 0.56992	train's my_metric_2: 0.0860662	valid's l2: 0.735035	valid's my_metric_2: 0.0931154
[102]	train's l2: 0.567607	train's my_metric_2: 0.0858528	valid's l2: 0.732553	valid's my_metric_2: 0.0929348
[103]	train's l2: 0.5

[186]	train's l2: 0.424027	train's my_metric_2: 0.0730864	valid's l2: 0.601195	valid's my_metric_2: 0.0830639
[187]	train's l2: 0.42263	train's my_metric_2: 0.0729845	valid's l2: 0.600217	valid's my_metric_2: 0.0830027
[188]	train's l2: 0.420756	train's my_metric_2: 0.0728294	valid's l2: 0.598163	valid's my_metric_2: 0.0828781
[189]	train's l2: 0.419592	train's my_metric_2: 0.0727208	valid's l2: 0.597413	valid's my_metric_2: 0.0828254
[190]	train's l2: 0.418047	train's my_metric_2: 0.0725839	valid's l2: 0.596143	valid's my_metric_2: 0.0826961
[191]	train's l2: 0.416995	train's my_metric_2: 0.0724917	valid's l2: 0.59532	valid's my_metric_2: 0.0826162
[192]	train's l2: 0.415762	train's my_metric_2: 0.0723844	valid's l2: 0.594027	valid's my_metric_2: 0.0825398
[193]	train's l2: 0.41498	train's my_metric_2: 0.0723151	valid's l2: 0.593761	valid's my_metric_2: 0.0825208
[194]	train's l2: 0.414087	train's my_metric_2: 0.0722271	valid's l2: 0.593305	valid's my_metric_2: 0.082478
[195]	train's 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

[53]	train's l2: 0.827043	train's my_metric_2: 0.106169	valid's l2: 0.848139	valid's my_metric_2: 0.110054
[54]	train's l2: 0.816675	train's my_metric_2: 0.105333	valid's l2: 0.838457	valid's my_metric_2: 0.109218
[55]	train's l2: 0.80658	train's my_metric_2: 0.104586	valid's l2: 0.829545	valid's my_metric_2: 0.108505
[56]	train's l2: 0.795768	train's my_metric_2: 0.103789	valid's l2: 0.817858	valid's my_metric_2: 0.107728
[57]	train's l2: 0.786604	train's my_metric_2: 0.103124	valid's l2: 0.809743	valid's my_metric_2: 0.107078
[58]	train's l2: 0.778867	train's my_metric_2: 0.102504	valid's l2: 0.803786	valid's my_metric_2: 0.106557
[59]	train's l2: 0.770311	train's my_metric_2: 0.101851	valid's l2: 0.796165	valid's my_metric_2: 0.105998
[60]	train's l2: 0.762328	train's my_metric_2: 0.101214	valid's l2: 0.788166	valid's my_metric_2: 0.105327
[61]	train's l2: 0.754889	train's my_metric_2: 0.100652	valid's l2: 0.781688	valid's my_metric_2: 0.104805
[62]	train's l2: 0.746704	train's my_m

[135]	train's l2: 0.503728	train's my_metric_2: 0.0796423	valid's l2: 0.574105	valid's my_metric_2: 0.0872488
[136]	train's l2: 0.501894	train's my_metric_2: 0.0794788	valid's l2: 0.572326	valid's my_metric_2: 0.0871122
[137]	train's l2: 0.500779	train's my_metric_2: 0.0793617	valid's l2: 0.571925	valid's my_metric_2: 0.0870643
[138]	train's l2: 0.498102	train's my_metric_2: 0.0791297	valid's l2: 0.569271	valid's my_metric_2: 0.0868337
[139]	train's l2: 0.496892	train's my_metric_2: 0.0790204	valid's l2: 0.568427	valid's my_metric_2: 0.0867699
[140]	train's l2: 0.495689	train's my_metric_2: 0.078905	valid's l2: 0.567727	valid's my_metric_2: 0.0866913
[141]	train's l2: 0.493451	train's my_metric_2: 0.0786984	valid's l2: 0.565733	valid's my_metric_2: 0.0865722
[142]	train's l2: 0.492368	train's my_metric_2: 0.0785746	valid's l2: 0.565255	valid's my_metric_2: 0.0864962
[143]	train's l2: 0.491388	train's my_metric_2: 0.0784876	valid's l2: 0.564538	valid's my_metric_2: 0.0864302
[144]	train

[28]	train's l2: 1.41399	train's my_metric_2: 0.140008	valid's l2: 1.35387	valid's my_metric_2: 0.140468
[29]	train's l2: 1.36456	train's my_metric_2: 0.137732	valid's l2: 1.30608	valid's my_metric_2: 0.138182
[30]	train's l2: 1.32136	train's my_metric_2: 0.135664	valid's l2: 1.26468	valid's my_metric_2: 0.136073
[31]	train's l2: 1.27963	train's my_metric_2: 0.133666	valid's l2: 1.22592	valid's my_metric_2: 0.134091
[32]	train's l2: 1.24258	train's my_metric_2: 0.131828	valid's l2: 1.19246	valid's my_metric_2: 0.132389
[33]	train's l2: 1.20751	train's my_metric_2: 0.13005	valid's l2: 1.16181	valid's my_metric_2: 0.130767
[34]	train's l2: 1.17583	train's my_metric_2: 0.128406	valid's l2: 1.13333	valid's my_metric_2: 0.129176
[35]	train's l2: 1.14527	train's my_metric_2: 0.126764	valid's l2: 1.10593	valid's my_metric_2: 0.127618
[36]	train's l2: 1.11718	train's my_metric_2: 0.125217	valid's l2: 1.0815	valid's my_metric_2: 0.126226
[37]	train's l2: 1.09081	train's my_metric_2: 0.12371	val

[111]	train's l2: 0.549492	train's my_metric_2: 0.083775	valid's l2: 0.597216	valid's my_metric_2: 0.0893157
[112]	train's l2: 0.547415	train's my_metric_2: 0.083588	valid's l2: 0.595661	valid's my_metric_2: 0.0891964
[113]	train's l2: 0.54492	train's my_metric_2: 0.0833608	valid's l2: 0.593697	valid's my_metric_2: 0.089015
[114]	train's l2: 0.543197	train's my_metric_2: 0.0832052	valid's l2: 0.592479	valid's my_metric_2: 0.0889095
[115]	train's l2: 0.540993	train's my_metric_2: 0.0830055	valid's l2: 0.590744	valid's my_metric_2: 0.0887687
[116]	train's l2: 0.539053	train's my_metric_2: 0.0828474	valid's l2: 0.589213	valid's my_metric_2: 0.0886454
[117]	train's l2: 0.536531	train's my_metric_2: 0.0826332	valid's l2: 0.587551	valid's my_metric_2: 0.0884968
[118]	train's l2: 0.534988	train's my_metric_2: 0.0824777	valid's l2: 0.586512	valid's my_metric_2: 0.0883891
[119]	train's l2: 0.533497	train's my_metric_2: 0.0823319	valid's l2: 0.585742	valid's my_metric_2: 0.0882976
[120]	train's 

mae: [0.08204059345161227, 0.08339344742602993, 0.08411866100764022, 0.08246350375570262, 0.08548353518186443, 0.08044556934389208, 0.08370410809043002, 0.08366111658674363, 0.08179221735554246, 0.08163378928935164] 0.08287365414888093


In [35]:
train_features = X_train_2.columns
model_2.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_2.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
6,时间戳,829
8,day,605
21,温度*压强,570
24,辐照度_差1,435
5,湿度,423
20,风速*温度,352
15,压强-mean_everyday,349
3,温度,331
1,风速,326
0,辐照度,325


In [36]:
from sklearn.metrics import mean_absolute_error
def my_metric_3(y_true,y_pred): 
    y_true_3 = y_true[y_true>=40*0.03]
    y_pred_3 = y_pred[y_true>=40*0.03]
    score = mean_absolute_error(y_true_3,y_pred_3)/40.0
    return 'my_metric_3',score,False  # False表示score不是越大越好(is_bigger_better)

In [37]:
X_train_3 = data_3[:len(train_3)].drop(['时间','实发辐照度','实际功率','id'],axis=1)

y_train_3 = data_3[:len(train_3)]['实际功率']

X_test_3 = data_3[len(train_3):].drop(['时间','实发辐照度','实际功率','id'],axis=1)


prediction_3 = test_3[['id']]

X_train_3_loc = X_train_3.values
X_test_3_loc = X_test_3.values
y_train_3_loc = y_train_3.values

model_3 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=200,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.01,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_train_3_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_3.fit(X_train_3_loc[train_index], y_train_3_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_3_loc[train_index], y_train_3_loc[train_index]), 
                                    (X_train_3_loc[test_index], y_train_3_loc[test_index])],
                          eval_metric=my_metric_3,
                          early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_3'])
    loss += lgb_model.best_score_['valid']['my_metric_3']
    test_pred= lgb_model.predict(X_test_3_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_3['predict_%s' % str(i)] = test_pred
print('mae:', baseloss, loss/10)

p = prediction_3.drop(['id'],axis=1)
prediction_3['prediction']=p.mean(axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 109.174	train's my_metric_3: 0.273658	valid's l2: 113.228	valid's my_metric_3: 0.278053
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 100.043	train's my_metric_3: 0.262395	valid's l2: 103.821	valid's my_metric_3: 0.266679
[3]	train's l2: 91.7989	train's my_metric_3: 0.251781	valid's l2: 95.3325	valid's my_metric_3: 0.256023
[4]	train's l2: 84.3506	train's my_metric_3: 0.24185	valid's l2: 87.6091	valid's my_metric_3: 0.245944
[5]	train's l2: 77.5673	train's my_metric_3: 0.232464	valid's l2: 80.5695	valid's my_metric_3: 0.236463
[6]	train's l2: 71.4519	train's my_metric_3: 0.223702	valid's l2: 74.2617	valid's my_metric_3: 0.227777
[7]	train's l2: 65.9293	train's my_metric_3: 0.215441	valid's l2: 68.5691	valid's my_metric_3: 0.219572
[8]	train's l2: 60.8943	train's my_metric_3: 0.207636	valid's l2: 63.409	valid's my_metric_3: 0.2118
[9]	train's l2: 56.3127	train's my_metric_3: 0.200315	valid's l2: 58.6844	valid's my_metric_3: 0.20438

[80]	train's l2: 6.61425	train's my_metric_3: 0.0701571	valid's l2: 8.07987	valid's my_metric_3: 0.0750092
[81]	train's l2: 6.54295	train's my_metric_3: 0.0697845	valid's l2: 8.03156	valid's my_metric_3: 0.0747526
[82]	train's l2: 6.49451	train's my_metric_3: 0.0694786	valid's l2: 7.99366	valid's my_metric_3: 0.0745491
[83]	train's l2: 6.43433	train's my_metric_3: 0.0691419	valid's l2: 7.95547	valid's my_metric_3: 0.0743587
[84]	train's l2: 6.36678	train's my_metric_3: 0.0687412	valid's l2: 7.89468	valid's my_metric_3: 0.0740004
[85]	train's l2: 6.31198	train's my_metric_3: 0.0683857	valid's l2: 7.84803	valid's my_metric_3: 0.0737143
[86]	train's l2: 6.25137	train's my_metric_3: 0.0679924	valid's l2: 7.79234	valid's my_metric_3: 0.0733815
[87]	train's l2: 6.18668	train's my_metric_3: 0.0675853	valid's l2: 7.74409	valid's my_metric_3: 0.0730885
[88]	train's l2: 6.12969	train's my_metric_3: 0.0672445	valid's l2: 7.69392	valid's my_metric_3: 0.0728059
[89]	train's l2: 6.08072	train's my_m

[177]	train's l2: 3.81974	train's my_metric_3: 0.0518309	valid's l2: 5.91815	valid's my_metric_3: 0.0626022
[178]	train's l2: 3.80531	train's my_metric_3: 0.0517308	valid's l2: 5.9081	valid's my_metric_3: 0.0625475
[179]	train's l2: 3.78523	train's my_metric_3: 0.0515986	valid's l2: 5.89247	valid's my_metric_3: 0.0624694
[180]	train's l2: 3.77443	train's my_metric_3: 0.0515098	valid's l2: 5.88544	valid's my_metric_3: 0.0624325
[181]	train's l2: 3.76094	train's my_metric_3: 0.0514152	valid's l2: 5.87634	valid's my_metric_3: 0.0623798
[182]	train's l2: 3.74638	train's my_metric_3: 0.0513004	valid's l2: 5.86922	valid's my_metric_3: 0.0623258
[183]	train's l2: 3.73002	train's my_metric_3: 0.0511816	valid's l2: 5.85526	valid's my_metric_3: 0.0622545
[184]	train's l2: 3.72064	train's my_metric_3: 0.0510956	valid's l2: 5.85104	valid's my_metric_3: 0.062214
[185]	train's l2: 3.70596	train's my_metric_3: 0.0509862	valid's l2: 5.83871	valid's my_metric_3: 0.0621336
[186]	train's l2: 3.69147	trai

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

[180]	train's l2: 3.66309	train's my_metric_3: 0.0506604	valid's l2: 6.89752	valid's my_metric_3: 0.067592
[181]	train's l2: 3.64545	train's my_metric_3: 0.0505492	valid's l2: 6.87255	valid's my_metric_3: 0.0674967
[182]	train's l2: 3.62864	train's my_metric_3: 0.0504273	valid's l2: 6.85491	valid's my_metric_3: 0.0674041
[183]	train's l2: 3.61675	train's my_metric_3: 0.0503357	valid's l2: 6.84602	valid's my_metric_3: 0.0673374
[184]	train's l2: 3.60132	train's my_metric_3: 0.05023	valid's l2: 6.82624	valid's my_metric_3: 0.0672298
[185]	train's l2: 3.58912	train's my_metric_3: 0.0501267	valid's l2: 6.81649	valid's my_metric_3: 0.0671516
[186]	train's l2: 3.57593	train's my_metric_3: 0.0500141	valid's l2: 6.79998	valid's my_metric_3: 0.0670657
[187]	train's l2: 3.56453	train's my_metric_3: 0.0499247	valid's l2: 6.78594	valid's my_metric_3: 0.067018
[188]	train's l2: 3.54854	train's my_metric_3: 0.0498065	valid's l2: 6.77465	valid's my_metric_3: 0.0669665
[189]	train's l2: 3.53552	train'

[63]	train's l2: 8.07224	train's my_metric_3: 0.078278	valid's l2: 9.4676	valid's my_metric_3: 0.0807228
[64]	train's l2: 7.94377	train's my_metric_3: 0.0775856	valid's l2: 9.36167	valid's my_metric_3: 0.0801649
[65]	train's l2: 7.85286	train's my_metric_3: 0.0770395	valid's l2: 9.26485	valid's my_metric_3: 0.0796435
[66]	train's l2: 7.74245	train's my_metric_3: 0.0764865	valid's l2: 9.14372	valid's my_metric_3: 0.0791702
[67]	train's l2: 7.6436	train's my_metric_3: 0.0759242	valid's l2: 9.06523	valid's my_metric_3: 0.0786914
[68]	train's l2: 7.54855	train's my_metric_3: 0.0754271	valid's l2: 8.96739	valid's my_metric_3: 0.0782691
[69]	train's l2: 7.44382	train's my_metric_3: 0.0748118	valid's l2: 8.86207	valid's my_metric_3: 0.0777492
[70]	train's l2: 7.34729	train's my_metric_3: 0.0742978	valid's l2: 8.77232	valid's my_metric_3: 0.0773475
[71]	train's l2: 7.27313	train's my_metric_3: 0.0738563	valid's l2: 8.71407	valid's my_metric_3: 0.0770116
[72]	train's l2: 7.18458	train's my_metr

[160]	train's l2: 3.98618	train's my_metric_3: 0.0531603	valid's l2: 5.90996	valid's my_metric_3: 0.0622404
[161]	train's l2: 3.96656	train's my_metric_3: 0.053053	valid's l2: 5.8978	valid's my_metric_3: 0.062176
[162]	train's l2: 3.95278	train's my_metric_3: 0.0529457	valid's l2: 5.88642	valid's my_metric_3: 0.0620874
[163]	train's l2: 3.93626	train's my_metric_3: 0.0528343	valid's l2: 5.87369	valid's my_metric_3: 0.0619929
[164]	train's l2: 3.91995	train's my_metric_3: 0.0527072	valid's l2: 5.86246	valid's my_metric_3: 0.0619188
[165]	train's l2: 3.90403	train's my_metric_3: 0.0525994	valid's l2: 5.85465	valid's my_metric_3: 0.0618979
[166]	train's l2: 3.89081	train's my_metric_3: 0.0524988	valid's l2: 5.8427	valid's my_metric_3: 0.0618208
[167]	train's l2: 3.87846	train's my_metric_3: 0.0523974	valid's l2: 5.83211	valid's my_metric_3: 0.0617357
[168]	train's l2: 3.86433	train's my_metric_3: 0.0522841	valid's l2: 5.81981	valid's my_metric_3: 0.0616622
[169]	train's l2: 3.84568	train'

[36]	train's l2: 13.533	train's my_metric_3: 0.103652	valid's l2: 16.6503	valid's my_metric_3: 0.113148
[37]	train's l2: 13.1359	train's my_metric_3: 0.102109	valid's l2: 16.2807	valid's my_metric_3: 0.111814
[38]	train's l2: 12.7692	train's my_metric_3: 0.100664	valid's l2: 15.9096	valid's my_metric_3: 0.110466
[39]	train's l2: 12.4196	train's my_metric_3: 0.0991949	valid's l2: 15.5647	valid's my_metric_3: 0.109129
[40]	train's l2: 12.0874	train's my_metric_3: 0.0977908	valid's l2: 15.2179	valid's my_metric_3: 0.107769
[41]	train's l2: 11.7814	train's my_metric_3: 0.0964518	valid's l2: 14.9548	valid's my_metric_3: 0.106618
[42]	train's l2: 11.4963	train's my_metric_3: 0.0951914	valid's l2: 14.663	valid's my_metric_3: 0.105456
[43]	train's l2: 11.2116	train's my_metric_3: 0.0939464	valid's l2: 14.3855	valid's my_metric_3: 0.104279
[44]	train's l2: 10.9558	train's my_metric_3: 0.0927894	valid's l2: 14.1379	valid's my_metric_3: 0.103201
[45]	train's l2: 10.7202	train's my_metric_3: 0.091

[131]	train's l2: 4.53849	train's my_metric_3: 0.0570395	valid's l2: 8.06607	valid's my_metric_3: 0.0725092
[132]	train's l2: 4.51841	train's my_metric_3: 0.0568873	valid's l2: 8.04798	valid's my_metric_3: 0.0723924
[133]	train's l2: 4.48858	train's my_metric_3: 0.056693	valid's l2: 8.00702	valid's my_metric_3: 0.0721627
[134]	train's l2: 4.46755	train's my_metric_3: 0.0565525	valid's l2: 7.98168	valid's my_metric_3: 0.0720414
[135]	train's l2: 4.44425	train's my_metric_3: 0.0563934	valid's l2: 7.95996	valid's my_metric_3: 0.0719346
[136]	train's l2: 4.42667	train's my_metric_3: 0.0562639	valid's l2: 7.94701	valid's my_metric_3: 0.0718403
[137]	train's l2: 4.4075	train's my_metric_3: 0.0561278	valid's l2: 7.92533	valid's my_metric_3: 0.0717185
[138]	train's l2: 4.38304	train's my_metric_3: 0.0559654	valid's l2: 7.90316	valid's my_metric_3: 0.0715881
[139]	train's l2: 4.36171	train's my_metric_3: 0.0558265	valid's l2: 7.86972	valid's my_metric_3: 0.071414
[140]	train's l2: 4.33417	train

In [38]:
train_features = X_train_3.columns
model_3.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_3.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
6,时间戳,837
8,day,750
21,温度*压强,650
20,风速*温度,620
3,温度,540
24,辐照度_差1,532
0,辐照度,480
5,湿度,460
13,湿度-mean_everyday,426
16,温度-mean_everyday,408


In [39]:
from sklearn.metrics import mean_absolute_error
def my_metric_4(y_true,y_pred): 
    y_true_4 = y_true[y_true>=50*0.03]
    y_pred_4 = y_pred[y_true>=50*0.03]
    score = mean_absolute_error(y_true_4,y_pred_4)/50.0
    return 'my_metric_4',score,False  # False表示score不是越大越好(is_bigger_better)

In [40]:
X_train_4 = data_4[:len(train_4)].drop(['时间','实发辐照度','实际功率','id'],axis=1)

y_train_4 = data_4[:len(train_4)]['实际功率']

X_test_4 = data_4[len(train_4):].drop(['时间','实发辐照度','实际功率','id'],axis=1)

prediction_4 = test_4[['id']]

X_train_4_loc = X_train_4.values
X_test_4_loc = X_test_4.values
y_train_4_loc = y_train_4.values

model_4 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=200,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.01,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_train_4_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_4.fit(X_train_4_loc[train_index], y_train_4_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_4_loc[train_index], y_train_4_loc[train_index]), 
                                    (X_train_4_loc[test_index], y_train_4_loc[test_index])],
                          eval_metric=my_metric_4,
                          early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_4'])
    loss += lgb_model.best_score_['valid']['my_metric_4']
    test_pred= lgb_model.predict(X_test_4_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_4['predict_%s' % str(i)] = test_pred
print('mae:', baseloss, loss/10)

p = prediction_4.drop(['id'],axis=1)
prediction_4['prediction']=p.mean(axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 206.217	train's my_metric_4: 0.298328	valid's l2: 206.906	valid's my_metric_4: 0.295207
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 188.746	train's my_metric_4: 0.285662	valid's l2: 189.573	valid's my_metric_4: 0.282825
[3]	train's l2: 172.951	train's my_metric_4: 0.273792	valid's l2: 173.965	valid's my_metric_4: 0.271334
[4]	train's l2: 158.641	train's my_metric_4: 0.262557	valid's l2: 159.847	valid's my_metric_4: 0.260442
[5]	train's l2: 145.672	train's my_metric_4: 0.252023	valid's l2: 147.139	valid's my_metric_4: 0.250364
[6]	train's l2: 133.905	train's my_metric_4: 0.242122	valid's l2: 135.635	valid's my_metric_4: 0.240945
[7]	train's l2: 123.266	train's my_metric_4: 0.232799	valid's l2: 125.145	valid's my_metric_4: 0.231907
[8]	train's l2: 113.671	train's my_metric_4: 0.224155	valid's l2: 115.734	valid's my_metric_4: 0.223583
[9]	train's l2: 104.936	train's my_metric_4: 0.216066	valid's l2: 107.159	valid's my_metric_4: 0.2

[84]	train's l2: 12.7902	train's my_metric_4: 0.0810976	valid's l2: 17.0376	valid's my_metric_4: 0.0909688
[85]	train's l2: 12.6999	train's my_metric_4: 0.0807607	valid's l2: 16.958	valid's my_metric_4: 0.0906967
[86]	train's l2: 12.6095	train's my_metric_4: 0.080401	valid's l2: 16.8786	valid's my_metric_4: 0.0903675
[87]	train's l2: 12.5351	train's my_metric_4: 0.0801729	valid's l2: 16.8056	valid's my_metric_4: 0.0901662
[88]	train's l2: 12.459	train's my_metric_4: 0.0798763	valid's l2: 16.7164	valid's my_metric_4: 0.0898748
[89]	train's l2: 12.3639	train's my_metric_4: 0.0795333	valid's l2: 16.6356	valid's my_metric_4: 0.0896368
[90]	train's l2: 12.286	train's my_metric_4: 0.0792468	valid's l2: 16.5804	valid's my_metric_4: 0.089447
[91]	train's l2: 12.2215	train's my_metric_4: 0.0790302	valid's l2: 16.5246	valid's my_metric_4: 0.0892808
[92]	train's l2: 12.1421	train's my_metric_4: 0.0787169	valid's l2: 16.4366	valid's my_metric_4: 0.0889718
[93]	train's l2: 12.0698	train's my_metric

[171]	train's l2: 8.99593	train's my_metric_4: 0.06657	valid's l2: 13.9007	valid's my_metric_4: 0.0803268
[172]	train's l2: 8.97452	train's my_metric_4: 0.0665001	valid's l2: 13.8862	valid's my_metric_4: 0.080291
[173]	train's l2: 8.93691	train's my_metric_4: 0.066341	valid's l2: 13.8537	valid's my_metric_4: 0.0801987
[174]	train's l2: 8.9173	train's my_metric_4: 0.0662522	valid's l2: 13.8364	valid's my_metric_4: 0.0801338
[175]	train's l2: 8.88663	train's my_metric_4: 0.0661063	valid's l2: 13.8053	valid's my_metric_4: 0.0800199
[176]	train's l2: 8.85531	train's my_metric_4: 0.065978	valid's l2: 13.7739	valid's my_metric_4: 0.0799179
[177]	train's l2: 8.83453	train's my_metric_4: 0.0658924	valid's l2: 13.7594	valid's my_metric_4: 0.0798695
[178]	train's l2: 8.81487	train's my_metric_4: 0.0658056	valid's l2: 13.7432	valid's my_metric_4: 0.0798243
[179]	train's l2: 8.7954	train's my_metric_4: 0.0657171	valid's l2: 13.7206	valid's my_metric_4: 0.0797494
[180]	train's l2: 8.7795	train's my

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

[160]	train's l2: 9.29733	train's my_metric_4: 0.0678502	valid's l2: 14	valid's my_metric_4: 0.0797746
[161]	train's l2: 9.27327	train's my_metric_4: 0.0677562	valid's l2: 13.9785	valid's my_metric_4: 0.0797072
[162]	train's l2: 9.24324	train's my_metric_4: 0.0676335	valid's l2: 13.9678	valid's my_metric_4: 0.0796422
[163]	train's l2: 9.20449	train's my_metric_4: 0.0674764	valid's l2: 13.9461	valid's my_metric_4: 0.0795765
[164]	train's l2: 9.17058	train's my_metric_4: 0.067342	valid's l2: 13.9118	valid's my_metric_4: 0.07949
[165]	train's l2: 9.14043	train's my_metric_4: 0.0672285	valid's l2: 13.8997	valid's my_metric_4: 0.0794404
[166]	train's l2: 9.1232	train's my_metric_4: 0.0671581	valid's l2: 13.895	valid's my_metric_4: 0.0794207
[167]	train's l2: 9.10139	train's my_metric_4: 0.0670731	valid's l2: 13.8952	valid's my_metric_4: 0.0793936
[168]	train's l2: 9.08103	train's my_metric_4: 0.0669844	valid's l2: 13.8826	valid's my_metric_4: 0.0793364
[169]	train's l2: 9.0472	train's my_me

[51]	train's l2: 18.0349	train's my_metric_4: 0.0983018	valid's l2: 20.1406	valid's my_metric_4: 0.105152
[52]	train's l2: 17.7558	train's my_metric_4: 0.0975027	valid's l2: 19.9017	valid's my_metric_4: 0.104474
[53]	train's l2: 17.4889	train's my_metric_4: 0.0967394	valid's l2: 19.6765	valid's my_metric_4: 0.103838
[54]	train's l2: 17.2464	train's my_metric_4: 0.0959742	valid's l2: 19.4805	valid's my_metric_4: 0.103221
[55]	train's l2: 17.0349	train's my_metric_4: 0.0953192	valid's l2: 19.2912	valid's my_metric_4: 0.102669
[56]	train's l2: 16.8111	train's my_metric_4: 0.0946284	valid's l2: 19.1278	valid's my_metric_4: 0.102152
[57]	train's l2: 16.5816	train's my_metric_4: 0.0939144	valid's l2: 18.9261	valid's my_metric_4: 0.101529
[58]	train's l2: 16.3817	train's my_metric_4: 0.0932753	valid's l2: 18.7634	valid's my_metric_4: 0.100995
[59]	train's l2: 16.1994	train's my_metric_4: 0.0926404	valid's l2: 18.5972	valid's my_metric_4: 0.10044
[60]	train's l2: 16.02	train's my_metric_4: 0.0

[135]	train's l2: 10.1914	train's my_metric_4: 0.0712434	valid's l2: 13.72	valid's my_metric_4: 0.0838893
[136]	train's l2: 10.1674	train's my_metric_4: 0.0711394	valid's l2: 13.7104	valid's my_metric_4: 0.0838503
[137]	train's l2: 10.1348	train's my_metric_4: 0.0710084	valid's l2: 13.6745	valid's my_metric_4: 0.083747
[138]	train's l2: 10.1083	train's my_metric_4: 0.0709006	valid's l2: 13.66	valid's my_metric_4: 0.0836771
[139]	train's l2: 10.0782	train's my_metric_4: 0.0707806	valid's l2: 13.6404	valid's my_metric_4: 0.0836037
[140]	train's l2: 10.0242	train's my_metric_4: 0.0705658	valid's l2: 13.6135	valid's my_metric_4: 0.0834916
[141]	train's l2: 9.99529	train's my_metric_4: 0.0704658	valid's l2: 13.5972	valid's my_metric_4: 0.0834108
[142]	train's l2: 9.95525	train's my_metric_4: 0.070303	valid's l2: 13.5604	valid's my_metric_4: 0.0832656
[143]	train's l2: 9.91992	train's my_metric_4: 0.0701841	valid's l2: 13.5422	valid's my_metric_4: 0.0832098
[144]	train's l2: 9.8832	train's m

[25]	train's l2: 37.7046	train's my_metric_4: 0.138284	valid's l2: 39.6201	valid's my_metric_4: 0.140152
[26]	train's l2: 35.9601	train's my_metric_4: 0.135545	valid's l2: 37.8376	valid's my_metric_4: 0.137547
[27]	train's l2: 34.3685	train's my_metric_4: 0.132981	valid's l2: 36.2188	valid's my_metric_4: 0.135139
[28]	train's l2: 32.9083	train's my_metric_4: 0.130564	valid's l2: 34.7218	valid's my_metric_4: 0.132927
[29]	train's l2: 31.5773	train's my_metric_4: 0.128241	valid's l2: 33.3816	valid's my_metric_4: 0.130773
[30]	train's l2: 30.35	train's my_metric_4: 0.126117	valid's l2: 32.1405	valid's my_metric_4: 0.128831
[31]	train's l2: 29.2075	train's my_metric_4: 0.124029	valid's l2: 30.9746	valid's my_metric_4: 0.12689
[32]	train's l2: 28.1611	train's my_metric_4: 0.122027	valid's l2: 29.9082	valid's my_metric_4: 0.125049
[33]	train's l2: 27.1911	train's my_metric_4: 0.120184	valid's l2: 28.8998	valid's my_metric_4: 0.123345
[34]	train's l2: 26.3069	train's my_metric_4: 0.118409	val

[130]	train's l2: 10.3216	train's my_metric_4: 0.0718246	valid's l2: 13.0952	valid's my_metric_4: 0.0810937
[131]	train's l2: 10.2785	train's my_metric_4: 0.0716519	valid's l2: 13.0599	valid's my_metric_4: 0.080974
[132]	train's l2: 10.2511	train's my_metric_4: 0.0715351	valid's l2: 13.0405	valid's my_metric_4: 0.0808963
[133]	train's l2: 10.2179	train's my_metric_4: 0.071404	valid's l2: 13.0199	valid's my_metric_4: 0.0808112
[134]	train's l2: 10.192	train's my_metric_4: 0.0713062	valid's l2: 12.9981	valid's my_metric_4: 0.0807359
[135]	train's l2: 10.159	train's my_metric_4: 0.0711726	valid's l2: 12.9681	valid's my_metric_4: 0.0806241
[136]	train's l2: 10.1267	train's my_metric_4: 0.071036	valid's l2: 12.9449	valid's my_metric_4: 0.0805373
[137]	train's l2: 10.1018	train's my_metric_4: 0.0709397	valid's l2: 12.9215	valid's my_metric_4: 0.0804549
[138]	train's l2: 10.0735	train's my_metric_4: 0.0708233	valid's l2: 12.8895	valid's my_metric_4: 0.080359
[139]	train's l2: 10.0382	train's 

In [41]:
train_features = X_train_4.columns
model_4.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_4.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
8,day,762
6,时间戳,616
20,风速*温度,592
21,温度*压强,582
5,湿度,467
3,温度,459
24,辐照度_差1,453
16,温度-mean_everyday,445
2,风向,435
12,辐照度-mean_everyday,429


In [42]:
prediction = pd.concat([prediction_1,prediction_2],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_3],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_4],axis=0,sort=False)
prediction

,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,prediction
0,1,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
1,2,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
2,3,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
3,4,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
4,5,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
5,6,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
6,7,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
7,8,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
8,9,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662
9,10,-0.005700,-0.005666,-0.005397,-0.005330,-0.005877,-0.005483,-0.006162,-0.004791,-0.005830,-0.006389,-0.005662


In [43]:
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')

prediction.rename(columns={'prediction':'predicition'},inplace=True)
prediction[['id', 'predicition']].to_csv("./result/baseline_%s.csv" % now, index=False)
